In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [84]:
data = pd.read_csv(r"C:\Users\Maria\Desktop\Программирование_3 сем\math\data_hw4.csv")
data.head()

,Item,Type,Group,Element,Controller,Word Number,Text,Sheet,Date Time,Timestamp,Device Id,Answer,Time Spent,Is Correct,Z-Score,Factor1,Factor2,Factor3
0,11,stimul1_light_think_0,1,0,AcceptabilityJudgment,NaN,"Дина думает, что Сева испачкал штаны.",0,25.05.2020 11:52,1590396746,588f4a3118f5ad5ae57d710af9510160,6.0,4564,NaN,0.764719,light,think,0.0
1,18,stimul1_light_say_1,2,0,AcceptabilityJudgment,NaN,"Что Паша говорит, что Вера нашла?",0,25.05.2020 11:52,1590396746,588f4a3118f5ad5ae57d710af9510160,3.0,2405,NaN,-0.611775,light,say,1.0
2,25,stimul1_say_whisper_0,3,0,AcceptabilityJudgment,NaN,"Митя шепчет, что Даша купила яблоки.",0,25.05.2020 11:52,1590396746,588f4a3118f5ad5ae57d710af9510160,3.0,3805,NaN,-0.611775,say,whisper,0.0
3,32,stimul1_say_babble_1,4,0,AcceptabilityJudgment,NaN,"Что Федя лепечет, что Тоня продала?",0,25.05.2020 11:52,1590396746,588f4a3118f5ad5ae57d710af9510160,1.0,2424,NaN,-1.529438,say,babble,1.0
4,39,stimul1_factive_regret_0,5,0,AcceptabilityJudgment,NaN,"Валя сожалеет, что Вова выпил молоко.",0,25.05.2020 11:52,1590396746,588f4a3118f5ad5ae57d710af9510160,6.0,6081,NaN,0.764719,factive,regret,0.0


In [85]:
data.drop(columns = ['Item', 'Type', 'Group', 'Element', 'Controller', 
'Word Number', 'Text', 'Sheet', 'Date Time', 'Timestamp', 'Device Id', 
'Time Spent', 'Factor2', 'Factor3', 'Is Correct'], axis = 1, inplace = True)

In [86]:
data = data.dropna()

In [87]:
data = data.loc[data['Factor1'] != '1']
data = data.loc[data['Factor1'] != '0']

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11431 entries, 0 to 23303
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Answer   11431 non-null  float64
 1   Z-Score  11431 non-null  float64
 2   Factor1  11431 non-null  object 
dtypes: float64(2), object(1)
memory usage: 357.2+ KB


In [89]:
data['Z-Score'].mean()

-0.05789833883194822

In [90]:
from scipy.stats import ttest_ind
ttest_ind(data['Z-Score'], 0)

TtestResult(statistic=-0.061054382088719654, pvalue=0.9513169666150705, df=11430.0)

моё лицо, когда p-value почти 1 ://////////

In [91]:
from scipy.stats import ttest_1samp
ttest_1samp(data['Z-Score'], 0)

TtestResult(statistic=-6.527966967272029, pvalue=6.949080864446885e-11, df=11430)

другое дело. p-value < 0,05 => гипотеза о равенстве нулю среднего по Z-score отвергается

In [92]:
#Тест Шапиро-Уилка
from scipy.stats import shapiro 

In [93]:
shapiro(data['Answer'])

C:\Users\Maria\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.8627836108207703, pvalue=0.0)

In [94]:
shapiro(data['Z-Score'])

ShapiroResult(statistic=0.9087499380111694, pvalue=0.0)

p < 0,05, можем отвергнуть нулевую гипотезу. 

Данные выборки не получены из нормального распределения.

In [95]:
#Тест Колмогорова-Смирнова
from scipy.stats import kstest

In [96]:
kstest(data['Answer'], 'norm')

KstestResult(statistic=0.8462027155717228, pvalue=0.0, statistic_location=2.0, statistic_sign=-1)

In [97]:
kstest(data['Z-Score'], 'norm')

KstestResult(statistic=0.11692778591870412, pvalue=1.2765737889050524e-136, statistic_location=-0.738481448, statistic_sign=1)

p < 0,05, нулевая гипотеза отвергается.

Данные выборки не получены из нормального распределения.
___

Односторонний анализ

In [98]:
from scipy.stats import f_oneway

In [99]:
light_a = data[data['Factor1'] == 'light']['Answer']
say_a = data[data['Factor1'] == 'say']['Answer']
factive_a = data[data['Factor1'] == 'factive']['Answer']

In [100]:
f_oneway(light_a, say_a, factive_a)

F_onewayResult(statistic=59.46870286815135, pvalue=2.025548579576222e-26)

In [101]:
import pingouin as pg

In [102]:
#анализ Уэлча
pg.welch_anova (dv='Answer', between='Factor1', data=data)

,Source,ddof1,ddof2,F,p-unc,np2
0,Factor1,2,7611.250973,62.42586,1.284552e-27,0.0103


In [103]:
light_z = data[data['Factor1'] == 'light']['Z-Score']
say_z = data[data['Factor1'] == 'say']['Z-Score']
factive_z = data[data['Factor1'] == 'factive']['Z-Score']

In [104]:
f_oneway(light_z, say_z, factive_z)

F_onewayResult(statistic=70.29706590328809, pvalue=4.535685382574082e-31)

In [105]:
pg.welch_anova (dv='Z-Score', between='Factor1', data=data)

,Source,ddof1,ddof2,F,p-unc,np2
0,Factor1,2,7605.660866,74.867815,6.327698e-33,0.012153


Двусторонний анализ

In [106]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [107]:
data.head()

,Answer,Z-Score,Factor1
0,6.0,0.764719,light
1,3.0,-0.611775,light
2,3.0,-0.611775,say
3,1.0,-1.529438,say
4,6.0,0.764719,factive


In [108]:
model = ols('Factor1 ~ Answer', data=data[data.Factor1 == 'light']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
Answer,1.0,5.879022e-31,5.879022e-31,0.001143,0.973029
Residual,3815.0,1.961907e-24,5.142613e-28,NaN,NaN


p > 0,05 => отсутсвие значимости

In [109]:
model = ols('Factor1 ~ Answer', data=data[data.Factor1 == 'say']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
Answer,1.0,8.237511e-29,8.237511e-29,426.942195,5.655432e-90
Residual,3796.0,7.324080e-28,1.929421e-31,NaN,NaN


p < 0,05 => статистически значимо

In [110]:
model = ols('Factor1 ~ Answer', data=data[data.Factor1 == 'factive']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
Answer,1.0,2.485616e-29,2.485616e-29,0.604132,0.437054
Residual,3814.0,1.569218e-25,4.114362e-29,NaN,NaN


p > 0,05 => отсутсвие значимости

In [111]:
model = ols('Factor1 ~ Q("Z-Score")', data=data[data.Factor1 == 'light']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
"Q(""Z-Score"")",1.0,6.396988e-30,6.396988e-30,5.077171,0.024299
Residual,3815.0,4.806714e-27,1.259951e-30,NaN,NaN


p < 0,05 => статистически значимо

In [112]:
model = ols('Factor1 ~ Q("Z-Score")', data=data[data.Factor1 == 'say']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
"Q(""Z-Score"")",1.0,1.090570e-30,1.090570e-30,2.456415,0.117129
Residual,3796.0,1.685303e-27,4.439680e-31,NaN,NaN


p > 0,05 => отсутсвие значимости

In [113]:
model = ols('Factor1 ~ Q("Z-Score")', data=data[data.Factor1 == 'factive']).fit()
sm.stats.anova_lm(model, typ=1)

,df,sum_sq,mean_sq,F,PR(>F)
"Q(""Z-Score"")",1.0,5.232782e-31,5.232782e-31,0.294661,0.587281
Residual,3814.0,6.773160e-27,1.775868e-30,NaN,NaN


p > 0,05 => отсутсвие значимости

In [114]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [120]:
tukey_light = pairwise_tukeyhsd(endog=data[data['Factor1'] == 'light']['Answer'][:100],
 groups=data[data['Factor1'] == 'light']['Z-Score'][:100],
 alpha= 0.05 )

print(tukey_light)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
   group1       group2    meandiff p-adj lower upper reject
-----------------------------------------------------------
-2.060704975 -1.702956675      0.0   nan   0.0   0.0  False
-2.060704975 -1.513883922      0.0   nan   0.0   0.0  False
-2.060704975 -1.235441536      1.0   0.0   1.0   1.0   True
-2.060704975 -1.198443035      1.0   0.0   1.0   1.0   True
-2.060704975 -1.187776505      1.0   0.0   1.0   1.0   True
-2.060704975 -1.171533251      0.0   nan   0.0   0.0  False
-2.060704975 -1.110595808     -1.0   0.0  -1.0  -1.0   True
-2.060704975 -1.070606758      0.0   nan   0.0   0.0  False
-2.060704975 -1.040192569     -1.0   0.0  -1.0  -1.0   True
-2.060704975  -1.01543951      0.0   nan   0.0   0.0  False
-2.060704975 -0.974281732      1.0   0.0   1.0   1.0   True
-2.060704975 -0.797135459      1.0   0.0   1.0   1.0   True
-2.060704975 -0.765576313      0.0   nan   0.0   0.0  False
-2.060704975 -0.685604125      2.0   0.0

In [117]:
tukey_say = pairwise_tukeyhsd(endog=data[data['Factor1'] == 'say']['Answer'][:100],
 groups=data[data['Factor1'] == 'say']['Z-Score'][:100],
 alpha= 0.05 )

print(tukey_say)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
   group1       group2    meandiff p-adj lower upper reject
-----------------------------------------------------------
-1.702956675 -1.529438226     -1.0   0.0  -1.0  -1.0   True
-1.702956675 -1.513883922      0.0   nan   0.0   0.0  False
-1.702956675 -1.466745958     -1.0   0.0  -1.0  -1.0   True
-1.702956675 -1.461073609      1.0   0.0   1.0   1.0   True
-1.702956675 -1.268773939      0.0   nan   0.0   0.0  False
-1.702956675 -1.235441536      1.0   0.0   1.0   1.0   True
-1.702956675 -1.198443035      1.0   0.0   1.0   1.0   True
-1.702956675 -1.187776505      1.0   0.0   1.0   1.0   True
-1.702956675  -1.17388368     -1.0   0.0  -1.0  -1.0   True
-1.702956675 -1.171533251      0.0   nan   0.0   0.0  False
-1.702956675 -1.110595808     -1.0   0.0  -1.0  -1.0   True
-1.702956675 -1.070606758      0.0   nan   0.0   0.0  False
-1.702956675 -1.040192569     -1.0   0.0  -1.0  -1.0   True
-1.702956675  -1.01543951      0.0   nan

In [122]:
tukey_factive = pairwise_tukeyhsd(endog=data[data['Factor1'] == 'factive']['Answer'][:100],
 groups=data[data['Factor1'] == 'factive']['Z-Score'][:100],
 alpha= 0.05 )

print(tukey_factive)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
   group1       group2    meandiff p-adj lower upper reject
-----------------------------------------------------------
-1.887547781 -1.702956675      0.0   nan   0.0   0.0  False
-1.887547781 -1.669610261     -1.0   0.0  -1.0  -1.0   True
-1.887547781 -1.513883922      0.0   nan   0.0   0.0  False
-1.887547781 -1.466745958     -1.0   0.0  -1.0  -1.0   True
-1.887547781 -1.461073609      1.0   0.0   1.0   1.0   True
-1.887547781 -1.286565765      1.0   0.0   1.0   1.0   True
-1.887547781 -1.268773939      0.0   nan   0.0   0.0  False
-1.887547781 -1.235441536      1.0   0.0   1.0   1.0   True
-1.887547781 -1.198443035      1.0   0.0   1.0   1.0   True
-1.887547781 -1.187776505      1.0   0.0   1.0   1.0   True
-1.887547781  -1.17388368     -1.0   0.0  -1.0  -1.0   True
-1.887547781 -1.110595808     -1.0   0.0  -1.0  -1.0   True
-1.887547781 -1.070606758      0.0   nan   0.0   0.0  False
-1.887547781 -1.040192569     -1.0   0.0

___

In [163]:
from scipy.stats import mannwhitneyu, kruskal, wilcoxon, friedmanchisquare

In [143]:
#тест Крускала-Уоллиса
kruskal(data['Answer'], light_a, say_a, factive_a)

KruskalResult(statistic=139.22620877123407, pvalue=5.550088007401262e-30)

In [144]:
kruskal(data['Z-Score'], light_z, say_z, factive_z)

KruskalResult(statistic=168.74304805624672, pvalue=2.376932765059778e-36)

p < 0,05 => статистически значимые различия

In [149]:
#тест Уилкоксона
wilcoxon(data[data['Factor1'] == 'light']['Answer'], data[data['Factor1'] == 'light']['Z-Score'])

WilcoxonResult(statistic=0.0, pvalue=0.0)

In [160]:
#U-тест Манна-Уитни
mannwhitneyu (data[data['Factor1'] == 'light']['Answer'], data[data['Factor1'] == 'light']['Z-Score'], alternative='two-sided')

MannwhitneyuResult(statistic=14269329.0, pvalue=0.0)

In [151]:
wilcoxon(data[data['Factor1'] == 'say']['Answer'], data[data['Factor1'] == 'say']['Z-Score'])

WilcoxonResult(statistic=0.0, pvalue=0.0)

In [161]:
mannwhitneyu (data[data['Factor1'] == 'say']['Answer'], data[data['Factor1'] == 'say']['Z-Score'], alternative='two-sided')

MannwhitneyuResult(statistic=14260130.0, pvalue=0.0)

In [152]:
wilcoxon(data[data['Factor1'] == 'factive']['Answer'], data[data['Factor1'] == 'factive']['Z-Score'])

WilcoxonResult(statistic=0.0, pvalue=0.0)

In [162]:
mannwhitneyu (data[data['Factor1'] == 'factive']['Answer'], data[data['Factor1'] == 'factive']['Z-Score'], alternative='two-sided')

MannwhitneyuResult(statistic=14179904.0, pvalue=0.0)